In [ ]:
# ! pip install fastparquet --user
# ! pip install ordered-set
# ! pip install swifter --user

In [ ]:
import pandas as pd
import swifter
import numpy as np
import gc
from tqdm import tqdm
from collections import defaultdict
from ordered_set import OrderedSet
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [ ]:
train = pd.read_parquet("../data_phase1/train.parquet")
val = pd.read_parquet("../data_phase1/validation.parquet")


In [ ]:
train = train.fillna(value={"context_type": "NA"})
val = val.fillna(value={"context_type": "NA"})


In [ ]:
train.describe(include='O')

In [ ]:
val.describe(include='O')

In [ ]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


In [ ]:
train = reduce_mem_usage(train)
val = reduce_mem_usage(val)


In [ ]:
gc.collect()

In [ ]:
train = train.sort_values(by='query_id').reset_index(drop=True)
val = val.sort_values(by='query_id').reset_index(drop=True)


In [ ]:
train.head(10)

In [ ]:
train.isnull().sum()

In [ ]:
data = pd.concat([train[list(set(train.columns)-set(['is_click']))], val], axis=0)
data = data.reset_index(drop=True)

In [ ]:
all_products = data.product_id.apply(str).tolist()

In [ ]:
unique_products = OrderedSet(all_products)


In [ ]:
train_session_interactions = dict(train.groupby('session_id')['product_id'].apply(list))
val_session_interactions = dict(val.groupby('session_id')['product_id'].apply(list))

train_session_actions = dict(train.groupby('session_id')['is_click'].apply(list))
# val_session_actions = dict(val.groupby('session_id')['is_click'].apply(list))


In [ ]:
train['days_elapsed'] = (train['week'].astype(int)-1)*7 + train['week_day'].astype(int)
val['days_elapsed'] = (val['week'].astype(int)-1)*7 + val['week_day'].astype(int)

In [ ]:
val['days_elapsed'] = val['days_elapsed'] - train['days_elapsed'].min()
train['days_elapsed'] = train['days_elapsed'] - train['days_elapsed'].min()


In [ ]:
val['days_elapsed'] = val['days_elapsed'] - train['days_elapsed'].min()
val.days_elapsed.unique()

In [ ]:
gc.collect()

In [ ]:
cols = [
 'query_id',
 'user_id',
 'session_id',
 'product_id',
 'user_tier',
 'week',
 'week_day',
 'days_elapsed'
]


In [ ]:
cols.append('is_click')
train = train[cols]

In [ ]:
attribute_data = pd.read_parquet("../data_phase1/attributes.parquet")

In [ ]:
train_attr_data = train.merge(attribute_data, on='product_id', how='left')


In [ ]:
is_click = train_attr_data.is_click.values
train_attr_data = train_attr_data.drop(['is_click'], 1)
train_attr_data['is_click'] = is_click


In [ ]:
train_attr_data.head(2)

In [ ]:
weekly_product_start_online_date = train_attr_data.groupby(['product_id', 'week']).agg({'start_online_date': 'mean'}).reset_index()
weekly_product_start_online_date["week"] = weekly_product_start_online_date["week"].astype(str)
weekly_product_start_online_date = pd.pivot_table(weekly_product_start_online_date, values=['start_online_date'], index=['product_id'],
                    columns=['week'], aggfunc=np.mean)
weekly_product_start_online_date.columns = weekly_product_start_online_date.columns.to_series().str.join('_')
weekly_product_start_online_date = weekly_product_start_online_date.reset_index()
weekly_product_start_online_date = weekly_product_start_online_date.rename(columns={
                                                             "start_online_date_1": "start_online_date_mean_clicked_out_week_1",
                                                             "start_online_date_2": "start_online_date_mean_clicked_out_week_2",
                                                             "start_online_date_3": "start_online_date_mean_clicked_out_week_3",
                                                             "start_online_date_4": "start_online_date_mean_clicked_out_week_4",
                                                             "start_online_date_5": "start_online_date_mean_clicked_out_week_5",
                                                             "start_online_date_6": "start_online_date_mean_clicked_out_week_6",
                                                             "start_online_date_7": "start_online_date_mean_clicked_out_week_7",
                                                             "start_online_date_8": "start_online_date_mean_clicked_out_week_8",})
weekly_product_start_online_date.head()


In [ ]:
# weekly_product_start_online_date.to_csv("../preprocessed_data/clickout_features/weekly_mean_product_start_online_date_clicked_out.csv", index=False)



In [ ]:
weekly_max_product_start_online_date = train_attr_data.groupby(['product_id', 'week']).agg({'start_online_date': 'max'}).reset_index()
weekly_max_product_start_online_date["week"] = weekly_max_product_start_online_date["week"].astype(str)
weekly_max_product_start_online_date = pd.pivot_table(weekly_max_product_start_online_date, values=['start_online_date'], index=['product_id'],
                    columns=['week'], aggfunc=np.mean)
weekly_max_product_start_online_date.columns = weekly_max_product_start_online_date.columns.to_series().str.join('_')
weekly_max_product_start_online_date = weekly_max_product_start_online_date.reset_index()
weekly_max_product_start_online_date = weekly_max_product_start_online_date.rename(columns={
                                                             "start_online_date_1": "start_online_date_max_clicked_out_week_1",
                                                             "start_online_date_2": "start_online_date_max_clicked_out_week_2",
                                                             "start_online_date_3": "start_online_date_max_clicked_out_week_3",
                                                             "start_online_date_4": "start_online_date_max_clicked_out_week_4",
                                                             "start_online_date_5": "start_online_date_max_clicked_out_week_5",
                                                             "start_online_date_6": "start_online_date_max_clicked_out_week_6",
                                                             "start_online_date_7": "start_online_date_max_clicked_out_week_7",
                                                             "start_online_date_8": "start_online_date_max_clicked_out_week_8",})
weekly_max_product_start_online_date.head()


In [ ]:
# weekly_max_product_start_online_date.to_csv("../preprocessed_data/clickout_features/weekly_max_product_start_online_date_clicked_out.csv", index=False)


In [ ]:
weekly_min_product_start_online_date = train_attr_data.groupby(['product_id', 'week']).agg({'start_online_date': 'min'}).reset_index()
weekly_min_product_start_online_date["week"] = weekly_min_product_start_online_date["week"].astype(str)
weekly_min_product_start_online_date = pd.pivot_table(weekly_min_product_start_online_date, values=['start_online_date'], index=['product_id'],
                    columns=['week'], aggfunc=np.mean)
weekly_min_product_start_online_date.columns = weekly_min_product_start_online_date.columns.to_series().str.join('_')
weekly_min_product_start_online_date = weekly_min_product_start_online_date.reset_index()
weekly_min_product_start_online_date = weekly_min_product_start_online_date.rename(columns={
                                                             "start_online_date_1": "start_online_date_min_clicked_out_week_1",
                                                             "start_online_date_2": "start_online_date_min_clicked_out_week_2",
                                                             "start_online_date_3": "start_online_date_min_clicked_out_week_3",
                                                             "start_online_date_4": "start_online_date_min_clicked_out_week_4",
                                                             "start_online_date_5": "start_online_date_min_clicked_out_week_5",
                                                             "start_online_date_6": "start_online_date_min_clicked_out_week_6",
                                                             "start_online_date_7": "start_online_date_min_clicked_out_week_7",
                                                             "start_online_date_8": "start_online_date_min_clicked_out_week_8",})
weekly_min_product_start_online_date.head()


In [ ]:
# weekly_min_product_start_online_date.to_csv("../preprocessed_data/clickout_features/weekly_min_product_start_online_date_clicked_out.csv", index=False)


In [ ]:
main_colour_wise_click_perc = train_attr_data.groupby(["main_colour"]).agg({"is_click": "count"}).reset_index()
sum_clicks = main_colour_wise_click_perc.is_click.sum()
main_colour_wise_click_perc["is_click"] = 100.0 * (main_colour_wise_click_perc["is_click"]/sum_clicks)
main_colour_wise_click_perc = main_colour_wise_click_perc.rename(columns={"is_click": "percentage_main_colour_click"})
main_colour_wise_click_perc.head()


In [ ]:
# main_colour_wise_click_perc.to_csv("../preprocessed_data/clickout_features/main_colour_click_percentage.csv", index=False)



In [ ]:
second_colour_wise_click_perc = train_attr_data.groupby(["second_colour"]).agg({"is_click": "count"}).reset_index()
sum_clicks = second_colour_wise_click_perc.is_click.sum()
second_colour_wise_click_perc["is_click"] = 100.0 * (second_colour_wise_click_perc["is_click"]/sum_clicks)
second_colour_wise_click_perc = second_colour_wise_click_perc.rename(columns={"is_click": "percentage_second_colour_click"})
second_colour_wise_click_perc.head()


In [ ]:
# second_colour_wise_click_perc.to_csv("../preprocessed_data/clickout_features/second_colour_click_percentage.csv", index=False)



In [ ]:
category_id_l1_wise_click_perc = train_attr_data.groupby(["category_id_l1"]).agg({"is_click": "count"}).reset_index()
sum_clicks = category_id_l1_wise_click_perc.is_click.sum()
category_id_l1_wise_click_perc["is_click"] = 100.0 * (category_id_l1_wise_click_perc["is_click"]/sum_clicks)
category_id_l1_wise_click_perc = category_id_l1_wise_click_perc.rename(columns={"is_click": "percentage_category_id_l1_click"})
category_id_l1_wise_click_perc.head()


In [ ]:
category_id_l1_wise_click_perc.to_csv("../preprocessed_data/clickout_features/category_id_l1_click_percentage.csv", index=False)


In [ ]:
category_id_l2_wise_click_perc = train_attr_data.groupby(["category_id_l2"]).agg({"is_click": "count"}).reset_index()
sum_clicks = category_id_l2_wise_click_perc.is_click.sum()
category_id_l2_wise_click_perc["is_click"] = 100.0 * (category_id_l2_wise_click_perc["is_click"]/sum_clicks)
category_id_l2_wise_click_perc = category_id_l2_wise_click_perc.rename(columns={"is_click": "percentage_category_id_l2_click"})
category_id_l2_wise_click_perc.head()


In [ ]:
# category_id_l2_wise_click_perc.to_csv("../preprocessed_data/clickout_features/category_id_l2_click_percentage.csv", index=False)


In [ ]:
category_id_l3_wise_click_perc = train_attr_data.groupby(["category_id_l3"]).agg({"is_click": "count"}).reset_index()
sum_clicks = category_id_l3_wise_click_perc.is_click.sum()
category_id_l3_wise_click_perc["is_click"] = 100.0 * (category_id_l3_wise_click_perc["is_click"]/sum_clicks)
category_id_l3_wise_click_perc = category_id_l3_wise_click_perc.rename(columns={"is_click": "percentage_category_id_l3_click"})
category_id_l3_wise_click_perc.head()


In [ ]:
# category_id_l3_wise_click_perc.to_csv("../preprocessed_data/clickout_features/category_id_l3_click_percentage.csv", index=False)


In [ ]:
brand_id_wise_click_perc = train_attr_data.groupby(["brand_id"]).agg({"is_click": "count"}).reset_index()
sum_clicks = brand_id_wise_click_perc.is_click.sum()
brand_id_wise_click_perc["is_click"] = 100.0 * (brand_id_wise_click_perc["is_click"]/sum_clicks)
brand_id_wise_click_perc = brand_id_wise_click_perc.rename(columns={"is_click": "percentage_brand_id_click"})
brand_id_wise_click_perc.head()


In [ ]:
# brand_id_wise_click_perc.to_csv("../preprocessed_data/clickout_features/brand_id_click_percentage.csv", index=False)


In [ ]:
season_year_wise_click_perc = train_attr_data.groupby(["season_year"]).agg({"is_click": "count"}).reset_index()
sum_clicks = season_year_wise_click_perc.is_click.sum()
season_year_wise_click_perc["is_click"] = 100.0 * (season_year_wise_click_perc["is_click"]/sum_clicks)
season_year_wise_click_perc = season_year_wise_click_perc.rename(columns={"is_click": "percentage_season_year_click"})
season_year_wise_click_perc.head()


In [ ]:
# season_year_wise_click_perc.to_csv("../preprocessed_data/clickout_features/season_year_click_percentage.csv", index=False)


In [ ]:
# last_product_day_elapsed = train.groupby(['product_id', 'days_elapsed']).last()

In [ ]:
train_attr_data.groupby([""])